# Appropriations Data Ingestion Program

## Load Packages:

In [166]:
import pandas as pd 
from openpyxl import load_workbook
from io import BytesIO
import sqlite3
import os
import numpy as np

## Create SQLite database:

In [75]:
db_path = 'Appropriations_log.db'
connection = sqlite3.connect(db_path)

cursor = connection.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS CIOs (
    CIO_Name TEXT NOT NULL PRIMARY KEY,
    Supplier_Number TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS CIO_Categories (
    Category_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    CIO_Name TEXT NOT NULL,
    Category TEXT NOT NULL,
    FOREIGN KEY (CIO_Name) REFERENCES CIOs(CIO_Name)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Round_Information (
    Full_Round_Name TEXT PRIMARY KEY,
    Date_RR_App_Closed TEXT,
    Year TEXT,
    Semester TEXT,
    Round TEXT    
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Application_Information (
    Application_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    CIO_Name TEXT,
    Round TEXT,
    Savings REAL,
    Income REAL,
    Expenses REAL,
    Balance REAL,
    Total_Amount_Requested REAL,
    Total_Amount_Approved_PreDeduction REAL,
    Surplus_Deduction REAL,
    Total_Amount_Approved_PostDeduction,
    Total_Amount_Allocated REAL,
    FOREIGN KEY (CIO_Name) REFERENCES CIOs(CIO_Name),
    FOREIGN KEY (Round) REFERENCES Round_Information(Full_Round_Name)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Event_Information (
    Event_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Application_ID INTEGER,
    Event_Name TEXT,
    Event_Description TEXT,
    Event_Purpose TEXT,
    Event_Dates TEXT,
    Impacted_Students TEXT,
    FOREIGN KEY (Application_ID) REFERENCES Application_Information(Application_ID)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Item_Information (
    Item_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Application_ID INTEGER,
    Item_Name TEXT,
    Item_Event TEXT,
    Price_Verification TEXT,
    Item_Price REAL,
    Quantity REAL,
    Amount_Requested REAL,
    Amount_Approved REAL,
    Return_Comments REAL,
    Item_Category TEXT,
    Primary_Reviewer TEXT,
    Secondary_Reviewer TEXT,
    CIO_Response TEXT,
    FOREIGN KEY (Application_ID) REFERENCES Application_Information(Application_ID)
)
''')

## Load Data:

In [191]:
def load_data(file_path, sheet):
    # Load the Excel file with row 3 (index 2) as headers
    df = pd.read_excel(file_path, header=2, sheet_name = sheet)
    
    # Rename the 'Unnamed: 0' column to 'Round'
    df.rename(columns={'Unnamed: 0': 'Round'}, inplace=True)
    
    # List of columns to drop
    columns_to_drop = [
        "Uncategorized Requested", "Uncategorized Approved", "Renewable Alternatives\nRequested", 
        "Renewable Alternatives Approved", "Food: $150/Sem Requested", "Food $150/Sem Approved", 
        "Food: Essential\nRequested", "Food: Essential\nApproved", "Guest Speakers Honoraria Requested", 
        "Guest Speakers Honoraria Approved", "Publicity & Advertising\nRequested", 
        "Publicity & Advertising\nApproved", "Services/Entry Fees/Printing\nRequested", 
        "Services/Entry Fees/Printing\nApproved", "Subscriptions\nRequested", "Subscriptions\nApproved", 
        "Lodging\nRequested", "Lodging Approved", "Gas\nRequested", "Gas\nApproved", "Rental Car\nRequested", 
        "Rental Car\nApproved", "Bus\nRequested", "Bus\nApproved", "Train\nRequested", "Train\nApproved", 
        "Airfare Requested", "Airfare Approved", "Public Transportation\nRequested", 
        "Public Transportation\nApproved", "Approved vs Allocated Difference\nRequested", 
        "Approved vs Allocated Difference\nApproved", "NOT FUNDABLE: Misc Expenses\nRequested", 
        "NOT FUNDABLE: Misc Expenses\nApproved", "Fundable: Other\nRequested", "Fundable: Other\nApproved", "Uncategorized\nRequested"
    ]
    
    # Drop specific columns and any column with "Unnamed" in the name
    df = df.drop(columns=[col for col in df.columns if "Unnamed" in col or col in columns_to_drop], errors='ignore')

    # Define a mapping from column index to new name
    rename_dict = {
        1: 'Savings',
        2: 'Income',
        3: 'Expenses',
        4: 'Balance',
        5: 'Event_Name',
        6: 'Event_Description',
        7: 'Event_Purpose',
        8: 'Event_Dates',
        9: 'Impacted_Students',
        10: 'Item_Name',
        11: 'Item_Event',
        12: 'Price_Verification',
        13: 'Item_Price',
        14: 'Quantity',
        15: 'Amount_Requested',
        16: 'Amount_Approved',
        17: 'Return_Comments',
        19: 'Primary_Reviewer',
        20: 'Secondary_Reviewer',
        21: 'CIO_Response',
        22: 'Total_Amount_Requested',
        23: 'Total_Amount_Approved_PreDeduction',
        25: 'Surplus_Deduction',
        26: 'Total_Amount_Approved_PostDeduction'
    }
    
    # Rename columns based on the mapping
    for idx, new_name in rename_dict.items():
        try:
            old_name = df.columns[idx]
            df.rename(columns={old_name: new_name}, inplace=True)
        except IndexError:
            print(f"Warning: No column at index {idx}. Skipping renaming for this index.")

    df = df.drop(columns="Net Non-SAF Balance.1", errors='ignore')

    # Step 1: Ensure numerical columns have consistent NaN values
    df = df.apply(pd.to_numeric, errors='ignore')

    # Step 2: Remove cells with "Range of rows in round" and adjacent cells below it
    # Find indices where "Range of rows in round" appears in the dataframe
    rows_to_modify = df[df.isin(["Range of rows in round"])].stack().index
    
    # Iterate over each found location to nullify the target cells
    for row, col in rows_to_modify:
        # Set the specified cell to NaN
        df.at[row, col] = pd.NA
        # Set the cell directly below it to NaN
        df.at[row + 1, col] = pd.NA
        # Set the cell below and to the right to NaN
        if col != df.columns[-1]:  # Ensure it doesn't go out of bounds
            df.at[row + 1, df.columns[df.columns.get_loc(col) + 1]] = pd.NA

    # Step 3: Standardize all missing values to np.nan
    df.replace({pd.NA: np.nan}, inplace=True)
    
    # Step 4: Remove rows where every column except Application_ID is NaN
    df = df.dropna(how='all', subset=[col for col in df.columns if col != 'Application_ID'])

    # Step 5: Remove specific rows with embedded header content
    header_patterns = ["Round Reported", "Description of Event/Activity", "Purpose of Event/Activity"]
    
    # Drop rows where any cell matches any of the header patterns
    df = df[~df.apply(lambda row: row.astype(str).str.contains('|'.join(header_patterns)).any(), axis=1)]

    # Step 6: Add Application_ID by creating a unique ID for each "Round" entry
    df['Application_ID'] = df['Round'].notnull().cumsum()

    # Step 7: Reorder columns to place Application_ID at the beginning
    df = df[['Application_ID'] + [col for col in df.columns if col != 'Application_ID']]


    # Return the DataFrame
    return(df)

In [264]:
def ingest_data(df, db_path, file_path):
    """
    Imports a pandas DataFrame into the specified SQLite database.
    
    Parameters:
    - db_path (str): Path to the SQLite database file.
    - df (pd.DataFrame): The DataFrame to import.
    """
    
    # Connect to SQLite database
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()
    
    try:
        # Insert CIOs
        CIO_Name = os.path.splitext(os.path.basename(file_path))[0]
        cursor.execute('''
        INSERT OR IGNORE INTO CIOs (CIO_Name) VALUES (?)
        ''', (CIO_Name,))
        
        # Insert Application_Information
        required_app_cols = [
            'Application_ID', 'Round', 'Savings', 'Income', 'Expenses',
            'Balance', 'Total_Amount_Requested', 'Total_Amount_Approved_PreDeduction',
            'Surplus_Deduction', 'Total_Amount_Approved_PostDeduction'
            # Note: 'CIO_Name' and 'Total_Amount_Allocated' are excluded
        ]
        
        if all(col in df.columns for col in required_app_cols):
            application_df = df[required_app_cols].drop_duplicates()
            for _, row in application_df.iterrows():
                application_id = row['Application_ID']
                cio_name = CIO_Name # Retrieve CIO_Name from variable/mapping
        
                try:
                    cursor.execute('''
                        INSERT OR IGNORE INTO Application_Information 
                        (Application_ID, CIO_Name, Round, Savings, Income, Expenses, Balance, 
                         Total_Amount_Requested, Total_Amount_Approved_PreDeduction, 
                         Surplus_Deduction, Total_Amount_Approved_PostDeduction, Total_Amount_Allocated)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        application_id,
                        cio_name,  # Use the variable here
                        row['Round'],
                        row['Savings'],
                        row['Income'],
                        row['Expenses'],
                        row['Balance'],
                        row['Total_Amount_Requested'],
                        row['Total_Amount_Approved_PreDeduction'],
                        row['Surplus_Deduction'],
                        row['Total_Amount_Approved_PostDeduction'],
                        None  # Set Total_Amount_Allocated to NULL
                    ))
                except Exception as e:
                    print(f"Error inserting Application ID {application_id}: {e}")
        else:
            print("Warning: One or more Application_Information columns are missing.")

        
        # Insert Event_Information
        event_df = df[['Application_ID', 'Event_Name', 'Event_Description', 'Event_Purpose', 
                      'Event_Dates', 'Impacted_Students']].drop_duplicates()
        for _, row in event_df.iterrows():
            cursor.execute('''
                INSERT INTO Event_Information 
                (Application_ID, Event_Name, Event_Description, Event_Purpose, 
                 Event_Dates, Impacted_Students)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                row['Application_ID'], row['Event_Name'], row['Event_Description'], 
                row['Event_Purpose'], row['Event_Dates'], row['Impacted_Students']
            ))
            
        
        # Insert Item_Information
        item_df = df[[
            'Application_ID', 'Item_Name', 'Item_Event', 'Price_Verification', 
            'Item_Price', 'Quantity', 'Amount_Requested', 'Amount_Approved', 
            'Return_Comments', 'Category', 'Primary_Reviewer', 
            'Secondary_Reviewer', 'CIO_Response'
        ]].drop_duplicates()
        for _, row in item_df.iterrows():
            cursor.execute('''
                INSERT INTO Item_Information 
                (Application_ID, Item_Name, Item_Event, Price_Verification, Item_Price, 
                 Quantity, Amount_Requested, Amount_Approved, Return_Comments, 
                 Item_Category, Primary_Reviewer, Secondary_Reviewer, CIO_Response)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                row['Application_ID'], row['Item_Name'], row['Item_Event'], 
                row['Price_Verification'], row['Item_Price'], row['Quantity'], 
                row['Amount_Requested'], row['Amount_Approved'], row['Return_Comments'], 
                row['Category'], row['Primary_Reviewer'], 
                row['Secondary_Reviewer'], row['CIO_Response']
            ))
        
        # Commit all changes
        connection.commit()
        print("Data inserted successfully.")
        
    except Exception as e:
        connection.rollback()
        print(f"An error occurred: {e}")
    
    finally:
        connection.close()

In [180]:
def ingest_data2(file_path):
    CIO_Name = os.path.splitext(os.path.basename(file_path))[0]
    print(f"Attempting to insert: {CIO_Name}")

    # Insert `CIO_Name` into the `CIOs` table
    #cursor.execute('''
    #INSERT OR IGNORE INTO CIOs (CIO_Name, Supplier_Number) VALUES (?, ?)
    #''', (CIO_Name, "Unknown"))  # Replace "Unknown" with a default or actual value if available

    cursor.execute('''
    INSERT OR IGNORE INTO CIOs (CIO_Name) VALUES (?)
    ''', (CIO_Name,))
    
    # Commit the transaction
    connection.commit()
    print(f"Inserted: {CIO_Name}")

## Run Code:

In [258]:
MootCourt_file = r'C:\Users\17708\OneDrive - University of Virginia\!CIO Stuff\Appropriations Committee\Appropriations Dashboard\Extramural Moot Court.xlsx'
ChakDeCville_file = r'C:\Users\17708\OneDrive - University of Virginia\!CIO Stuff\Appropriations Committee\Appropriations Dashboard\Chak De Cville.xlsx'

db_path = 'Appropriations_log.db'

In [276]:
df = load_data(MootCourt_file, "F24")
df2 = load_data(ChakDeCville_file, "F24")

C:\Users\17708\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\17708\AppData\Local\Temp\ipykernel_17708\1897250996.py:66: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
C:\Users\17708\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\17708\AppData\Local\Temp\ipykernel_17708\1897250996.py:66: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [280]:
#Only want for loop here, everything else abstracted into functions
ingest_data(df, db_path, MootCourt_file)
ingest_data(df2, db_path, ChakDeCville_file)

Data inserted successfully.
Data inserted successfully.


C:\Users\17708\AppData\Local\Temp\ipykernel_17708\405475700.py:66: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''


## Check database:

In [286]:
# CIOs, CIO_Categories, Round_Information, Application_Information, Event_Information, Item_Information
cursor.execute('SELECT * FROM Application_Information')
rows = cursor.fetchall()

if rows:
    for row in rows:
        print(row)
else:
    print("Table Empty.")

(1, 'Extramural Moot Court', 'RR1', 10000.0, 0.0, 10000.0, 0.0, 5845.0, 5845.0, 0.0, 5845, None)
(2, 'Extramural Moot Court', 'RR2', 10000.0, 0.0, 10000.0, 0.0, 2200.0, 2200.0, 0.0, 2200, None)
(3, 'Extramural Moot Court', 'RR3', 10000.0, 0.0, 10000.0, 0.0, 4677.21, 4677.21, 0.0, 4677.21, None)


In [25]:
cursor.execute("PRAGMA table_info(CIOs)")
print(cursor.fetchall())

[(0, 'CIO_Name', 'TEXT', 1, None, 1), (1, 'Supplier_Number', 'TEXT', 0, None, 0)]
